# Artifact of Table 3 for Phase Moducation Side Channel Analysis and JTAG Attacks

This is part of a series of artifacts. Note these notebooks are designed to run on Google Colab so have been simplified from those available in the [full repository](https://github.com/colinoflynn/phase-modulation-sca).

To run these notebooks, simply run from top to bottom.

**WARNING**: Some steps, such as the extraction, can be very slow. Try not to forget you started the process as CoLab will kill unattended notebooks eventually, and you'll have to start again.

In [ ]:
!sudo apt-get install zstd

In [ ]:
!wget https://zenodo.org/records/13359322/files/ioleakage.tar.zst?download=1

In [ ]:
!tar -xf ioleakage.tar.zst?download=1

In [ ]:
!rm -rf  ioleakage.tar.zst?download=1

## T-Test Results

The following recreats the T-Test results for Table 3.

In [ ]:
import os
import numpy as np
def plot_t(t_val, N, titledata=""):
    import matplotlib.pylab as plt
    ldata = len(t_val)
    plt.plot(t_val)
    plt.plot([0, ldata], [4.5, 4.5], 'k--')
    plt.plot([0, ldata], [-4.5, -4.5], 'k--')
    plt.title("TVLA Results, N=%d, %s"%(N, titledata))
    plt.xlabel("Sample Number")
    plt.ylabel("T-Test Value")

#Set root data dir, not "mixers"
data_dir = ""

def npload(filename):
    return np.load(os.path.join(data_dir, filename), allow_pickle=True)

def nptload(prefix):
    group1 = npload(prefix + "_tvla_group1.npy")
    group2 = npload(prefix + "_tvla_group2.npy")
    N = round((len(group1) + len(group2)) / 2)
    return group1, group2, N

def npsave(filename, array):
    np.save(os.path.join(data_dir, filename), array)

def nptsave(prefix, group1, group2):
    npsave(prefix + "_tvla_group1.npy", group1)
    npsave(prefix + "_tvla_group2.npy", group2)


In [ ]:
files = [
         [r"ioleakage/gnd_clkin_120mhz_sync_120msps_ttest_10k", 100, 10000, 0.01, 0.1],
         [r"ioleakage/iohigh_clkin_120mhz_sync_120msps_ttest_10k", 100, 10000, 0.01, 0.1],
         [r"ioleakage/iolow_clkin_120mhz_sync_120msps_ttest_10k", 100, 10000, 0.01, 0.1],
         [r"ioleakage/icisolator_iolow_clkin_120mhz_sync_120msps_ttest_10k", 100, 10000, 0.01, 0.1],
         [r"ioleakage/opticalisolator_iolow_clkin_120mhz_sync_120msps_ttest_10k", 100, 10000,0.01, 0.1],
        ]

In [ ]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt
from scipy import signal

def find_max_t(file, start=0, end=-1, hpf=None, lpf=None, resync_jtag=False):
    group1, group2, N = nptload(file)

    if resync_jtag:
        first = []
        for g in group1:
            first.append(g[0])
        avg = np.mean(first)
        group1resync = []
        for g in group1:
            if g[0] > avg:
                group1resync.append(g[:-1])
            else:
                group1resync.append(g[1:])

        group2resync = []
        for g in group2:
            if g[0] > avg:
                group2resync.append(g[:-1])
            else:
                group2resync.append(g[1:])

        group1 = group1resync
        group2 = group2resync

    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    t_val = ttest_ind(group1[start:end], group2[start:end], axis=0, equal_var=False)[0]
    max_t = max(abs(t_val))
    return max_t, t_val
for f in files:
    max_t, t_val = find_max_t(f[0], start=f[1], end=f[2], hpf=f[3], lpf=f[4])
    print("{:s}: {:f}".format(f[0], max_t))

The output should look like this:

```
ioleakage/gnd_clkin_120mhz_sync_120msps_ttest_10k: 3.753704
ioleakage/iohigh_clkin_120mhz_sync_120msps_ttest_10k: 3.741362
ioleakage/iolow_clkin_120mhz_sync_120msps_ttest_10k: 5.113831
ioleakage/icisolator_iolow_clkin_120mhz_sync_120msps_ttest_10k: 1.970434
ioleakage/opticalisolator_iolow_clkin_120mhz_sync_120msps_ttest_10k: 3.503088
```

These can be compared to Table 3 in the paper:

| Measurement | Isolator | TVLA   |
|-------------|----------|--------|
| GND         | None     | 3.8    |
| IO-High     | None     | 3.7    |
| IO-Low      | None     | 5.1    |
| IO-Low      | IC-Based | 2.0    |
| IO-Low      | Optical  | 3.5    |

This should verify that the T-Test results of the IO-pin leakage match.


This verifies Table 3 and completes this artifact.